# <center>پیدا کردن ربات</center>

<div dir="rtl" style="font-size:18px">
احتمالا کارتون WALL-E را بخاطر دارید. رباتی که توانست زمین را نجات دهد و باقیمانده نسل انسان‌ها را به زمین برگرداند. پس از بازگشت انسان‌ها به زمین و پس از محبوبیت زیاد WALL-E، او دوستان زیادی پیدا کرد. بهترین دوست او WALL-F یکی از بازیگوش‌ترین ربات‌هایی است که تاحالا دیده‌اید.
<br>
یکی از دفعاتی که WALL-F به بیرون رفته بود، راهی به درون یک هزار‌تو میابد. تصویر زیر از او در این هزارتو گرفته شده است.
</div>
<br>
<br>
<center>

![WALL-F Ai-Generated](Assets/robot.jpg)

</center>

<div dir="rtl">
در ادامه میخواهیم با روش‌های مختلفی پیدا کنیم که در حال حاضر ربات در چه خانه‌ای قرار دارد. در هرکدام از تست‌کیس‌هایی که لازم است بررسی کنید. خانه 0 و 0 خانه بالا و چپ هزارتو خواهد بود. در ادامه برای هر خانه مشخص خواهد شد که دیواری به هرکدام از جهات دارد یا خیر. 
به طور مثال عبارت
''R L''
به این معنی است که این خانه فقط به چپ و راست خود راه دارد. همچنین شماره‌گذاری خانه‌ها به صورت زیر است.
در هر مرحله او به شما میگوید که در کدام خانه از جدول است. عددی که ربات به شما میگوید یک عدد تصادفی است که در یک ماتریس به شما داده میشود. عدد سطر 
$i$ 
و ستون 
$j$ 
احتمال این را نشان می‌دهد که ربات در خانه 
$i$ 
باشد اما خانه 
$j$ 
را شما گزارش دهد. این ربات در پایان هر مرحله با احتمال برابر به یکی از خانه‌های مجاور میرود. همینطور خانه شروع این ربات هم توزیعی یکنواخت دارد.
</div>
<br>
<br>
<center>

![grid](Assets/grid.jpg)

</center>

<div dir="rtl">
در ابتدا برای پیاده‌سازی ۲ الگوریتمی که در ادامه خواهید دید به آبجکتی از نوع Grid نیاز خواهید داشت. عملکرد این آبجکت را میتوانید از مسیر Assets/helpers.py ببینید. به طور مثال هزارتو زیر را در نظر بگیرید.
</div>

<center>

![maze](Assets/grid2.jpg)

</center>

<div dir="rtl">
grid متناظر با این هزارتو را میتوانید از طریق تابع get_sample_grid بگیرید و با توابع آن آشنا شوید. در صورت لزوم میتوانید هر تابع دلخواهی به این کلاس اضافه کنید. برای اطلاع از تعداد سطرهای grid میتوانید از grid.n و برای اطلاع از تعداد ستون‌ها میتوانید از grid.m استفاده کنید.
</div>

In [11]:
from Assets.helpers import get_sample_grid

In [12]:
grid = get_sample_grid()

# Particle Filtering

<div dir="rtl">
در این قسمت باید آخرین خانه‌ای که این ربات در آن قرار دارد را با استفاده از الگوریتم Particle Filtering پیدا کنید. برای اینکار در ابتدا تعداد Particle هایی با تعدادی از اردر تعداد خانه‌های هزارتو بسازید. در پایان هر مرحله جمعیت را با توجه به وزن آنها نصف کنید و ذراتی جدید متناسب با تعداد ذرات درون هرخانه بسازید. در پایان خانه‌ای که بیشترین ذرات در آن قرار دارد را برگردانید.
</div>

In [33]:
import numpy as np 
import copy
def PF(grid, prob, obs):
    """
    input:

    grid: object of type Grid
    obs: array of ints, each one is an observation

    output: 
    number of the last cell
    """

    num_cells = grid.n * grid.m
    num_samples = num_cells * 10

    particles = np.random.choice(num_cells, size=num_samples, replace=True)

    for ev in obs:
        
        # elapse
        particles = np.array([grid.get_neighbors(tile)[np.random.choice(len(grid.get_neighbors(tile)))]
                              for tile in particles])
        
        # update weights
        weights = [prob[p][ev] for p in particles]
        weights /= np.sum(weights)

        # resample
        replace_indices = np.random.choice(num_samples, size=num_samples // 2, replace=False)
        particles_cpy = particles.copy()
        particles[replace_indices] = np.random.choice(particles, size=num_samples // 2, replace=True, p=weights)
        # indices = np.random.choice(np.arange(num_samples), size=num_samples, replace=True, p=weights)
        # particles = particles[indices]

    return np.argmax(np.bincount(particles))
    pass

In [39]:
from Assets.helpers import PF_checker
PF_checker(PF)

PF PASSED!


# HMM

<div dir="rtl">
در این قسمت غیر از اینکه باید بفهمیم WALL-F در کدام خانه قرار دارد. میخواهیم بدانیم که محتمل‌ترین مسیری که طی کرده‌ است چه مسیری است. برای اینکار الگوریتم viterbi را پیاده‌سازی کنید.
</div>

In [15]:
def viterbi(grid, prob, obs):
    """
    input:

    grid: object of type Grid
    obs: array of ints, each one is an observation

    output: 
    array of cell numbers, same shape as obs
    """
    num_cells = grid.n * grid.m
    num_obsv = len(obs)
    
    viterbi = [{}]
    sequence = {}

    for state in range(num_cells):
      viterbi[0][state] = prob[state][obs[0]] / num_cells
      sequence[state] = [state]
    
    for o in range(1, num_obsv):
      viterbi.append({})
      path = {}

      for state in range(num_cells):
        pr = -1
        t = -1
        for s0 in grid.get_neighbors(state):
          tmp = viterbi[o - 1][s0] * prob[state][obs[o]] / len(grid.get_neighbors(s0))
          if tmp > pr:
            pr = tmp
            t = s0
        viterbi[o][state] = pr
        path[state] = sequence[t] + [state]
      
      sequence = path
    n = 0
    if len(obs) != 1:
      n = num_obsv - 1
    (pr, state) = max((viterbi[n][y], y) for y in range(num_cells))
    return sequence[state]



    pass

In [16]:
from Assets.helpers import viterbi_checker
viterbi_checker(viterbi)

VITERBI PASSED!
